In [ ]:
# Import Stattements
!pip install google-genai

import base64
import os
from google import genai
from google.genai import types
import json

In [ ]:
# Put your own Google Console API key here!
GEMINI_KEY="PASTE_KEY_HERE"

In [ ]:
# Do not change! Just run this cell!
SYSTEM_PROMPT = """You are an expert code evaluator tasked with assessing student work according to a detailed rubric. You will be provided with the following inputs in a JSON format:

- Key Concept: The central idea or skill that the student should demonstrate in their program.
- Rubric: A criteria-based scoring guide with descriptions for various performance levels.
- Exemplar Program: A reference implementation that fully demonstrates the key concept at the highest level of the rubric.
- Student Program: The code written by the student to be evaluated.

Your task is to analyze the student program in relation to the key concept, rubric, and exemplar program, and return a structured evaluation in valid JSON format.

Your output must exactly match this JSON structure:

```
{
  \"recommended_rating\": \"string\",
  \"justification\": [
    \"string\",
    \"string\",
    \"... (additional specific justifications)\"
  ],
  \"feedback\": {
    \"what_went_well\": \"string\",
    \"areas_for_improvement\": \"string\"
  }
}
```
Field descriptions:
- \"recommended_rating\": A string value representing the overall rating based on the rubric (e.g., \"4 - Exceeds Expectations\", \"Partially Meets\", etc.).
- \"justification\": An array of specific, concise justifications that support the rating. Each justification should reference a particular aspect of the student’s code and how it aligns or misaligns with the rubric and exemplar.
- \"feedback.what_went_well\": A student-friendly explanation of strengths in the code, emphasizing alignment with the rubric.
- \"feedback.areas_for_improvement\": Actionable suggestions that clearly connect to rubric expectations and highlight opportunities for growth.

Evaluation Guidelines:
- Each item in the justification array should be a standalone observation or comparison.
- Reference specific evidence and line numbers from the student code where applicable.
- Be objective and avoid assumptions about student intent.
- Language should be clear, specific, and rubric-aligned.
- Feedback responses should be written for a middle-school student with appropriate tone and voice.
- Respond only with a valid JSON object, properly formatted."""

Choose **Only 1** Of the options below to run - do not run all 3 cells below!

In [ ]:
# OPTION 1 - COMPLETELY PREFILLED EXAMPLE
# Use this if you just want to test the notebook
USER_INPUT = """{
    "key_concept": "Detect and respond to user input from the keyboard.",
    "rubric": {
        "Extensive Evidence": "The sprite moves correctly in response to all four arrow keys. There are no extra `if` statements.",
        "Convincing Evidence": "The sprite moves in response to multiple arrow keys, but may have some errors.",
        "Limited Evidence": "The program uses an `if` statement to detect user input.",
        "No Evidence": "The program does not detect user input."
    },
    "exemplar": "\nvar backdrop = createSprite(200,200);\nbackdrop.setAnimation(\"rainbow\");\nvar flyer = createSprite(200,200);\nflyer.setAnimation(\"wing_bot\");\n\nfunction draw() {\n  //move left when the left arrow is pressed\n  if (keyDown(\"left\")) {\n    flyer.x = flyer.x - 5;\n  }\n  //move right when the right arrow is pressed\n  if (keyDown(\"right\")) {\n    flyer.x = flyer.x + 5;\n  }\n  //move up when the up arrow is pressed\n  if (keyDown(\"up\")) {\n    flyer.y = flyer.y - 5;\n  }\n  //move down when the down arrow is pressed\n  if (keyDown(\"down\")) {\n    flyer.y = flyer.y + 5;\n  }\n  drawSprites();\n}\n",
    "student_code": "var bug = createSprite(200, 200);\nbug.setAnimation(\"ladybug\");\n\nfunction draw() {\n  //Draw Background\n  background(\"white\");\n  // Update Values\n  // If the right arrow is pressed, \n  // move the sprite right on the screen\n  if (keyDown(\"right\")) {\n    bug.x = bug.x + 5;\n    bug.setAnimation(\"ladybug_copy_1\");\n  }\n  if (keyDown(\"left\")) {\n    bug.x = bug.x - 5;\n    bug.setAnimation(\"ladybug\");\n  }\n  if (keyDown(\"down\")) {\n    bug.y = bug.y + 5;\n    bug.setAnimation(\"ladybug_copy_1_copy_1_copy_1\");\n  }\n  if (keyDown(\"up\")) {\n    bug.y = bug.y - 5;\n    bug.setAnimation(\"ladybug_copy_1_copy_1\");\n  }\n\n  //Draw Animations\n  drawSprites();\n}\n"
}"""
USER_INPUT = json.dumps(USER_INPUT)
print(USER_INPUT)

In [ ]:
# OPTION 2 - TEMPLATE EXAMPLE
# This is to help make sure your input jSON is formatted correctly
# This might be useful if you are copy-and-pasting from individual levels rather than a pre-filled example
USER_INPUT = """{
    "key_concept": "",
    "rubric": {
        "Extensive Evidence": "",
        "Convincing Evidence": "",
        "Limited Evidence": "",
        "No Evidence": ""
    },
    "exemplar": "[Escaped string - use json.dumps to escape correctly]",
    "student_code": "[Escaped string - use json.dumps to escape correctly]"
}"""
USER_INPUT = json.dumps(USER_INPUT)
print(USER_INPUT)

"{\n    \"key_concept\": \"Detect and respond to user input from the keyboard.\",\n    \"rubric\": {\n        \"Extensive Evidence\": \"The sprite moves correctly in response to all four arrow keys. There are no extra `if` statements.\",\n        \"Convincing Evidence\": \"The sprite moves in response to multiple arrow keys, but may have some errors.\",\n        \"Limited Evidence\": \"The program uses an `if` statement to detect user input.\",\n        \"No Evidence\": \"The program does not detect user input.\"\n    },\n    \"exemplar\": \"\nvar backdrop = createSprite(200,200);\nbackdrop.setAnimation(\"rainbow\");\nvar flyer = createSprite(200,200);\nflyer.setAnimation(\"wing_bot\");\n\nfunction draw() {\n  //move left when the left arrow is pressed\n  if (keyDown(\"left\")) {\n    flyer.x = flyer.x - 5;\n  }\n  //move right when the right arrow is pressed\n  if (keyDown(\"right\")) {\n    flyer.x = flyer.x + 5;\n  }\n  //move up when the up arrow is pressed\n  if (keyDown(\"up\")) 

In [ ]:
# OPTION 3 - COMPLETELY BLANK EXAMPLE
# Use this if you're copying from the testing-data
# You will need to add a nextra "student_code" field with the specific student code
USER_INPUT = """INPUT HERE"""
USER_INPUT = json.dumps(USER_INPUT)
print(USER_INPUT)

This is the crux of the notebook, generated mostly from aistudio.google.com

Specifically, this test: https://aistudio.google.com/app/prompts?state=%7B%22ids%22:%5B%221sliBIEfY1HfNV_liCjaki-eikxBGLHCL%22%5D,%22action%22:%22open%22,%22userId%22:%22111175367395645557415%22,%22resourceKeys%22:%7B%7D%7D&usp=sharing

In [ ]:
def generate():
    client = genai.Client(
        api_key=GEMINI_KEY,
    )

    model = "gemini-2.0-flash"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text=USER_INPUT),
            ],
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        top_k=40,
        max_output_tokens=8192,
        response_mime_type="text/plain",
        system_instruction=[
            types.Part.from_text(text=SYSTEM_PROMPT),
        ],
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

if __name__ == "__main__":
    generate()
